In [88]:
import logging
import re
import os
os.chdir('code/')
import random

import torch
from torch.utils.data import DataLoader
from tqdm import tqdm, trange
from transformers import (AdamW, AutoModelForSeq2SeqLM,
                          AutoTokenizer, get_linear_schedule_with_warmup)


FileNotFoundError: [WinError 2] The system cannot find the file specified: 'code/'

In [20]:
from constants import *
from data_utils import (ABSADataset, filter_none, filter_invalid,
                        get_dataset, get_inputs, normalize_augment)
from model_utils import (prepare_constrained_tokens, prepare_tag_tokens)
from main import *
from data_utils import *

In [3]:
os.getcwd()

'c:\\Workspace\\Tutorial\\BGCA\\code'

In [4]:
# Load arguments from the JSON file

import json
import argparse

arg_path = '../outputs/aste/cross_domain/run_aste/seed-42/laptop14-rest14/args.json'
with open(arg_path, 'r') as file:
    args_dict = json.load(file)

# Create a namespace from the dictionary
args = argparse.Namespace(**args_dict)

args.model_name_or_path = '../outputs/aste/cross_domain/run_aste/seed-42/laptop14-rest14/checkpoint-e24'
print(args)


Namespace(adam_epsilon=1e-08, beam=1, clear_model=True, commit=None, data_dir='../outputs/aste/cross_domain/run_aste/seed-42/laptop14-rest14/data', data_gene=True, data_gene_aug_num=None, data_gene_aug_ratio=None, data_gene_decode=None, data_gene_epochs=25, data_gene_extract=True, data_gene_extract_epochs=25, data_gene_extract_none_remove_ratio=0, data_gene_min_length=0, data_gene_none_remove_ratio=0, data_gene_none_word_num=1, data_gene_num_beam=1, data_gene_same_model=False, data_gene_top_p=0.9, data_gene_wt_constrained=True, dataset='cross_domain', device='cuda', do_eval=True, do_train=True, eval_batch_size=16, extract_model=None, gene_model=None, gradient_accumulation_steps=2, inference_dir='../outputs/aste/cross_domain/run_aste/seed-42/laptop14-rest14/inference', init_tag='english', learning_rate=0.0003, max_seq_length=128, model_filter=True, model_filter_skip_none=False, model_name_or_path='../outputs/aste/cross_domain/run_aste/seed-42/laptop14-rest14/checkpoint-e24', n_gpu='0', 

In [5]:
model = AutoModelForSeq2SeqLM.from_pretrained(args.model_name_or_path).to(args.device)
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=False)

In [25]:
test_sents, _ = read_line_examples_from_file(f"{args.data_dir}/test.txt")

for x, y in zip(test_sents, _):
    print(x, y)
    break
    


['The', 'bread', 'is', 'top', 'notch', 'as', 'well', '.'] [([1], [3, 4], 'POS')]


In [33]:
args.data_dir
os.getcwd()

'c:\\Workspace\\Tutorial\\BGCA\\code'

In [42]:

test_dataset = get_dataset(args, task=args.task, data_type="test", tokenizer=tokenizer)

inputs, targets = get_inputs_and_targets(args, task=args.task, data_type="test")
# inputs, targets = get_inputs_and_targets(args, task=args.task, data_type='thai_data/laptop14/th/googletrans/test_aste_processed.txt')

print(f'get_inputs_and_targets() --> {inputs[0]} ==> {targets[0]}')

# test_dataset[0]

get_inputs_and_targets() --> The bread is top notch as well . ==> <pos> bread <opinion> top notch


In [84]:
th_input = ['เวลาบูตเร็วสุด ๆ จาก 35 วินาทีถึง 1 นาที']
th_target = ['<pos> เวลาบูต <opinion> เร็ว']
th_input = ['เวลา บูต เร็ว สุด ๆ จาก 35 วินาที ถึง 1 นาที', 'เป็นเรื่องดีมากที่ไม่ต้องกังวลเกี่ยวกับเรื่องนี้และค่าใช้จ่ายเพิ่มเติมที่มาพร้อมกับการป้องกันไวรัสที่จำเป็นบนพีซี']
th_target = ['','']
en_input = ['The bread is top notch as well .']
en_target = ['<pos> bread <opinion> top notch']

In [86]:
# temp_dataset = ABSADataset(args, tokenizer, inputs=inputs, targets=targets)
temp_dataset = ABSADataset(args, tokenizer, inputs=th_input, targets=th_target)

# temp_dataset = ABSADataset(args, tokenizer, inputs=en_input, targets=en_target)

temp_dataset[1]

{'source_ids': tensor([   259, 158088, 150961,   1549,  83119, 169222,  34518, 117301,   2091,
         119228,  58618,   1549, 220596, 237063, 182034,   1549, 130025,  17383,
          57716,  25851,      1,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              

In [87]:
def infer_new(args, dataset, model, tokenizer, name, is_constrained=False, constrained_vocab=None, keep_mask=False, **decode_dict):
    dataloader = DataLoader(dataset, batch_size=args.eval_batch_size, num_workers=4)

    if keep_mask:
        # can't skip special directly, will lose extra_id
        unwanted_tokens = [tokenizer.eos_token, tokenizer.unk_token, tokenizer.pad_token]
        unwanted_ids = tokenizer.convert_tokens_to_ids(unwanted_tokens)
        def filter_decode(ids):
            ids = [i for i in ids if i not in unwanted_ids]
            tokens = tokenizer.convert_ids_to_tokens(ids)
            sentence = tokenizer.convert_tokens_to_string(tokens)
            return sentence

    # inference
    inputs, outputs, targets = [], [], []
    
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader):
            if is_constrained:
                prefix_fn_obj = Prefix_fn_cls(tokenizer, constrained_vocab, batch['source_ids'].to(args.device))  # need fix
                prefix_fn = lambda batch_id, sent: prefix_fn_obj.get(batch_id, sent)
            else:
                prefix_fn = None

            outs_dict = model.generate(input_ids=batch['source_ids'].to(args.device),
                                        attention_mask=batch['source_mask'].to(args.device),
                                        max_length=128,
                                        prefix_allowed_tokens_fn=prefix_fn,
                                        output_scores=True,
                                        return_dict_in_generate=True,
                                        **decode_dict,
                                        )
            outs = outs_dict["sequences"]

            if keep_mask:
                input_ = [filter_decode(ids) for ids in batch["source_ids"]]
                dec = [filter_decode(ids) for ids in outs]
                target = [filter_decode(ids) for ids in batch["target_ids"]]
            else:
                input_ = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["source_ids"]]
                dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
                target = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["target_ids"]]

            inputs.extend(input_)
            outputs.extend(dec)
            targets.extend(target)

    # decode_txt = "constrained" if is_constrained else "greedy"
    # with open(os.path.join(args.inference_dir, f"{name}_{decode_txt}_output.txt"), "w") as f:
    #     for i, o in enumerate(outputs):
    #         f.write(f"{inputs[i]} ===> {o}\n")

    # return inputs, outputs, targets
    return inputs, outputs


In [83]:
input_infer, output_infer = infer_new(
        args, temp_dataset, model, tokenizer, name=f"wtf",
        # is_constrained=True, 
        is_constrained=False, 
        constrained_vocab=prepare_constrained_tokens(tokenizer, args.task, args.paradigm),
    )


for x,y in zip(input_infer, output_infer):
    print(f'input text:{x}')
    print(f'predict(gene) text:{y}')

100%|██████████| 1/1 [00:04<00:00,  4.41s/it]

input text:เวลา บูต เร็ว สุด ๆ จาก 35 วินาที ถึง 1 นาที
predict(gene) text:<neu> บูต <opinion> เร็ว
input text:เป็นเรื่องดีมากที่ไม่ต้องกังวลเกี่ยวกับเรื่องนี้และค่าใช้จ่ายเพิ่มเติมที่มาพร้อมกับการป้องกันไวรัสที่จําเป็นบนพีซี
predict(gene) text:<pos> การป้องกัน virus <opinion> ดีมาก
